In [38]:
import random
import torch
from tqdm import tqdm
from functools import partialmethod
import gradio as gr
from gradio.mix import Series
from transformers import pipeline, FSMTForConditionalGeneration, FSMTTokenizer
from rudalle.pipelines import generate_images
from PIL import Image

In [10]:
# disable tqdm logging from the rudalle pipeline
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
translation_model = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-en-ru", torch_dtype=torch.float16).to(
    device)
translation_tokenizer = FSMTTokenizer.from_pretrained("facebook/wmt19-en-ru")
dalle = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)
tokenizer = get_tokenizer()
vae = get_vae().to(device)

◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.
tokenizer --> ready
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
vae --> ready


In [ ]:
def prediction(text):  
    text = ''
    pil_images = []
    scores = []
    
    seed_everything(6955)

    for top_k, top_p, images_num in [
    (1024, 0.98, 3),
    (512, 0.97, 3),
    (384, 0.96, 3), 
]:
        _pil_images, _scores = generate_images(text, tokenizer, dalle, vae, top_k=top_k, images_num=images_num, top_p=top_p)
        pil_images += _pil_images
        scores += _scores
    top_images, clip_scores = cherry_pick_by_clip(pil_images, text, ruclip, ruclip_processor, device=device, count=6)
    show(top_images, 3)

In [54]:
def translation_wrapper(text: str):
    input_ids = translation_tokenizer.encode(text, return_tensors="pt")
    outputs = translation_model.generate(input_ids.to(device))
    decoded = translation_tokenizer.decode(outputs[0].float(), skip_special_tokens=True)
    return decoded


def dalle_wrapper(prompt: str):
    
    pil_images = []
    
    top_k, top_p = random.choice([
        (1024, 0.98),
        (512, 0.97)
    ])

    _images, _ = generate_images(
        prompt,
        tokenizer,
        dalle,
        vae,
        top_k=top_k,
        images_num=1,
        top_p=top_p
    )
    pil_images += _images
    
    
    
    return pil_images

In [55]:
dalle_wrapper('Пудж на миде')

  0%|          | 0/1024 [00:00<?, ?it/s]

[<PIL.Image.Image image mode=RGB size=256x256 at 0x157FCE0E9D0>]

In [ ]:
,
        (384, 0.96),

In [8]:
def main():
    st.title('Text generation')
    menu = ['Сгенерировать контент','О проекте']
    choice = st.sidebar.selectbox('Menu',menu)
    if choice == 'Сгенерировать контент':
        st.subheader('Сгенерировать контент')
        with st.form(key='123'):
            raw_text = st.text_area('Введите тут')
            submit_text = st.form_submit_button(label='Сгенерировать!')

    

    else:
        st.subheader('О проекте')

if __name__ =='__main__':
    main()